# Separación para análisis por persona

In [21]:
import os
import sys
from collections import defaultdict
import cv2
import numpy as np
import time
import json
import matplotlib.pyplot as plt
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

ruta_utils = "../src/utils"

# Agregar la ruta al sys.path
if ruta_utils not in sys.path:
    sys.path.append(ruta_utils)

# Ahora puedes importar utils
import utils

In [22]:
model_path = "../src/models/yolo11n.pt"
video_path = "../data/robo1.mp4"
output_path = "../data/output_robo1_yolo.mp4"

# Load the YOLO model
model = YOLO(model_path)

# Open the video file and get properties
cap = utils.initialize_video_capture(video_path)
w, h, fps = utils.get_video_properties(cap)

# Initialize video writer
out = utils.initialize_video_writer(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, w, h)

In [23]:
# Cargar el modelo YOLO
model = YOLO(model_path)

# Inicializar la captura de video
cap = cv2.VideoCapture(video_path)

# Diccionario para almacenar los frames de cada persona
person_frames = {}

# Procesar el video frame por frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detectar personas en el frame con seguimiento
    results = model.track(frame, persist=True, verbose=False)

    # Procesar las detecciones
    for result in results:
        for box in result.boxes:
            # Verificar si es una persona (clase 0)
            if int(box.cls.item()) == 0:
                # Obtener coordenadas del bounding box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # Obtener ID de seguimiento
                person_id = getattr(box, 'id', None)
                if person_id is None:
                    continue
                person_id = int(person_id)

                # Validar y ajustar coordenadas
                height, width = frame.shape[:2]
                x1 = max(0, min(x1, width - 1))
                y2 = min(y2, height - 1)
                y1 = max(0, min(y1, y2 - 1))  # Asegurar y1 < y2
                x2 = max(x1 + 1, min(x2, width - 1))  # Asegurar x1 < x2

                # Recortar región válida
                if x1 >= x2 or y1 >= y2:
                    continue
                person_frame = frame[y1:y2, x1:x2]

                # Almacenar en el diccionario
                if person_id in person_frames:
                    person_frames[person_id].append(person_frame)
                else:
                    if len(person_frames) < 3:
                        person_frames[person_id] = [person_frame]

# Liberar recursos
cap.release()

# Mostrar resultados
for person_id, frames in person_frames.items():
    print(f"Persona {person_id}: {len(frames)} frames")

Persona 1: 340 frames
Persona 37: 2 frames


In [26]:
def save_person_video(person_id, person_frames, output_path='person_video.avi', fps=30):
    """
    Guarda un video con todos los frames de una persona detectada
    
    Args:
        person_id (int): ID de la persona a guardar
        person_frames (dict): Diccionario con los frames de cada persona
        output_path (str): Ruta de salida para el video
        fps (int): Cuadros por segundo del video resultante
    """
    # Validar entrada
    if person_id not in person_frames:
        raise ValueError(f"Persona ID {person_id} no encontrada en los datos")
        
    frames = person_frames[person_id]
    
    if not frames:
        raise ValueError("No hay frames para guardar")
    
    # Crear directorio si no existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Obtener dimensiones del primer frame
    height, width = frames[0].shape[:2]
    
    # Configurar codec y VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Puedes cambiar a 'MJPG' o 'MP4V' si prefieres
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if not out.isOpened():
        raise RuntimeError("No se pudo inicializar el VideoWriter. Verifica el codec y la ruta")
    
    # Escribir frames
    try:
        for frame in frames:
            # Verificar dimensiones y tipo de dato
            if frame.shape[:2] != (height, width):
                frame = cv2.resize(frame, (width, height))
                
            if frame.dtype != np.uint8:
                frame = frame.astype(np.uint8)
                
            out.write(frame)
            
    except Exception as e:
        out.release()
        os.remove(output_path)
        raise RuntimeError(f"Error escribiendo frames: {str(e)}")
    
    # Liberar recursos
    out.release()
    
    # Verificar que el archivo se creó
    if not os.path.exists(output_path):
        raise RuntimeError("Falló la creación del archivo de video")
    
    print(f"Video guardado exitosamente en: {output_path}")
    return True

In [27]:
# Ejemplo de uso
if person_frames:
    save_person_video(
        person_id=1,
        person_frames=person_frames,
        output_path='videos/persona_0.avi',
        fps=30
    )

Video guardado exitosamente en: videos/persona_0.avi
